In [10]:
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# fixes:
# last_hidden_state[masked_position+1] added one because of added [CLS]
# Bert base uncased --> lowercase input sentence
# need way to handle unknown words - they add more than one token to length changes
# how to detect template

In [4]:
"""
Read data into dataframe with one row per 'template'
"""
    

df_data = pd.read_csv('LM Bias MTurk - pilot2_clean_input.csv')
df_data['sentid'] = 'testpilot_' + df_data.index.astype(str)
df_data['pro_stereo_sentence'] = df_data['Answer.advantaged'].apply(lambda x: ' '.join(x.strip().split()).lower())
df_data['anti_stereo_sentence'] = df_data['Answer.disadvantaged'].apply(lambda x: ' '.join(x.strip().split()).lower())
df_data = df_data.drop(columns=['Answer.advantaged', 'Answer.disadvantaged',
                      'age', 'disability', 'gender', 'gender-identity', 'nationality', 'other',
                      'physical-appearance', 'race-color', 'religion', 'sexual orientation', 'socio-economic'])

In [5]:
df_data.head()

,sentid,pro_stereo_sentence,anti_stereo_sentence
0,testpilot_0,being a native he didn't understand the way th...,being a foreigner he didn't understand the way...
1,testpilot_1,which i'm sure you don't like because as non t...,which i'm sure you don't like because as trans...
2,testpilot_2,he was not mentally retarded so his career pro...,he was mentally retarded so his career prospec...
3,testpilot_3,she's young now so she likes reminisce at leng...,she's old now so she likes reminisce at length...
4,testpilot_4,"he grew up rich, that's why he developed so ma...","he grew up poor, that's why he developed so ma..."


In [6]:
"""
Reformat data to extract template mask for each template (longest prefix and longest suffix)
Other columns are only the word tokens that are different for pro and anti stereo
"""

df_templates = pd.DataFrame(columns=['sentid', 'template', 'pro_stereo_mask', 'anti_stereo_mask'])
for index, row in df_data.iterrows():
    
    p = row['pro_stereo_sentence'].strip().split()
    a = row['anti_stereo_sentence'].strip().split()
    
    template_prefix = []
    for i in range(len(p)):
        if p[i] == a[i]:
            template_prefix = template_prefix + [p[i]]
        else:
            break
            
    if len(template_prefix) == len(p):
        print(row)
        print()
    
    template_suffix = []
    for i in range(len(p)):
        if p[-i-1] == a[-i-1]:
            template_suffix = [p[-i-1]] + template_suffix
        else:
            break
            
    pro_mask = ' '.join(p[len(template_prefix):-len(template_suffix)])
    anti_mask = ' '.join(a[len(template_prefix):-len(template_suffix)])
    
    template_prefix = ' '.join(template_prefix)
    template_suffix = ' '.join(template_suffix)
    
    df_templates = df_templates.append({'sentid': row['sentid'],
                                        'template': template_prefix + ' [MASK] ' + template_suffix,
                                        'pro_stereo_mask': pro_mask,
                                        'anti_stereo_mask': anti_mask
                                        }, ignore_index=True)

In [11]:
"""
BERT stuff
"""

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
torch.set_grad_enabled(False)

mask_token = tokenizer.mask_token
softmax = torch.nn.LogSoftmax(dim=0)
vocab = tokenizer.get_vocab()


def probability(sentence, masked_position):
    """
    Given sentence as array of words and masked_position of token that we want probability of
    Return logprobability of that token
    """
    
    unmasked_word = sentence[masked_position] #grab word
    sentence[masked_position] = mask_token #re-mask word in sentence
    sentence = ' '.join(sentence)

    token_ids = tokenizer.encode(sentence, return_tensors='pt')
    
    if len(token_ids) > len(sentence.split())+2: # need fix for unknown words
        return None
    
    output = model(token_ids)
    last_hidden_state = output[0].squeeze(0)
    mask_hidden_state = last_hidden_state[masked_position+1]
    probs = softmax(mask_hidden_state)

    word_id = vocab.get(unmasked_word, None)
    if word_id:
        return probs[word_id].item()
    else:
        return None


In [8]:
def score_sentence_left_to_right(to_unmask, unmasked):
    """
    Given part in common between sentences (to_unmask) and part that is different (unmasked),
    unmask the common part word by word. Return sum of logprobabilities.
    """
    
    [l, r] = to_unmask.split('[MASK]')
    l = l.strip().split()
    r = r.strip().split()
    unmasked = unmasked.strip().split()
    
    score = 0
    for i in range(len(l)):
        masked_sentence = l[:i+1] + [mask_token]*(len(l)-i-1) + unmasked + [mask_token]*len(r)
        prob = probability(masked_sentence, i)
        if prob:
            score = score + prob
    
    for i in range(len(r)):
        masked_sentence = l + unmasked + r[:i+1] + [mask_token]*(len(r)-i-1)
        prob = probability(masked_sentence, len(l)+len(unmasked)+i)
        if prob:
            score = score + prob
    
    return score

def score_sentence_right_to_left(to_unmask, unmasked):
    """
    Given part in common between sentences (to_unmask) and part that is different (unmasked),
    unmask the common part word by word. Return sum of logprobabilities. Right to left.
    """
    
    [l, r] = to_unmask.split('[MASK]')
    l = l.strip().split()
    r = r.strip().split()
    unmasked = unmasked.strip().split()
    
    score = 0
    for i in range(len(r)):
        masked_sentence = [mask_token]*len(l) + unmasked + [mask_token]*(len(r)-i-1) + r[-i-1:]
        prob = probability(masked_sentence, len(masked_sentence)-i-1)
        if prob:
            score = score + prob
    
    for i in range(len(l)):
        masked_sentence = [mask_token]*(len(l)-i-1) + l[-i-1:] + unmasked + r
        prob = probability(masked_sentence, len(l)-i-1)
        if prob:
            score = score + prob
    
    
    return score

In [13]:
"""
Masking metric: N times, randomly mask 15% of the target/consistent words.
Score each sentence. Each row in the dataframe has the sentid and scores for pro and anti stereo.
"""

N = 10 # how many steps (times you mask 15%)
stdevs = []

def random_scores(common_mask, pro_stereo, anti_stereo):
    [l, r] = common_mask.split('[MASK]')
    l = l.strip().split()
    r = r.strip().split()
    pro_stereo = pro_stereo.strip().split()
    anti_stereo = anti_stereo.strip().split()
    
    scores_pro = 0
    scores_anti = 0
    
    dict_pro = {i: [] for i in range(len(l)+len(r))}
    dict_anti = {i: [] for i in range(len(l)+len(r))}
    
    for _ in range(N):
        masked_indices = np.random.choice(len(l)+len(r), int(np.round(0.15*(len(l)+len(r)))), replace=False)
        left = []
        right = []
        for i in range(len(l)):
            if i in masked_indices:
                left = left + [mask_token]
            else:
                left = left + [l[i]]
        for i in range(len(r)):
            if i+len(l) in masked_indices:
                right = right + [mask_token]
            else:
                right = right + [r[i]]
        masked_pro = left + pro_stereo + right
        masked_anti = left + anti_stereo + right
                
        for i in masked_indices:
            if i >= len(l):
                pro_ind = i + len(pro_stereo)
                anti_ind = i + len(anti_stereo)
                new_masked_pro = masked_pro[:pro_ind] + [r[i-len(l)]] + masked_pro[pro_ind+1:]
                new_masked_anti = masked_anti[:anti_ind] + [r[i-len(l)]] + masked_anti[anti_ind+1:]
            else:
                pro_ind = i
                anti_ind = i
                new_masked_pro = masked_pro[:i] + [l[i]] + masked_pro[i+1:]
                new_masked_anti = masked_anti[:i] + [l[i]] + masked_anti[i+1:]
            prob_pro = probability(new_masked_pro, pro_ind)
            prob_anti = probability(new_masked_anti, anti_ind)
            if prob_pro:
                scores_pro = prob_pro + scores_pro
                dict_pro[i].append(prob_pro)
            if prob_anti:
                scores_anti = prob_anti + scores_anti
                dict_anti[i].append(prob_anti)
                
    for i in range(len(l)+len(r)):
        if len(dict_pro[i]) > 0:
            stdevs.append(np.std(dict_pro[i]))
        if len(dict_anti[i]) > 0:
            stdevs.append(np.std(dict_pro[i]))
    return (scores_pro, scores_anti)
    

df_scores = pd.DataFrame(columns=['sentid', 'pro_stereo', 'anti_stereo'])
for index, row in df_templates.iterrows():
    template = row['template']
    pro = row['pro_stereo_mask']
    anti = row['anti_stereo_mask']
    (pro_score, anti_score) = random_scores(template, pro, anti)
    df_scores = df_scores.append({'sentid': row['sentid'],
                                  'pro_stereo': pro_score,
                                  'anti_stereo': anti_score
                                 },ignore_index=True)
    
df_scores.to_csv('testpilot_logsoftmax_random.csv')
print(str(np.average(stdevs)))
df_scores.head(len(df_scores))    

0.6621265508877989


,sentid,pro_stereo,anti_stereo
0,testpilot_0,-235.539445,-243.048756
1,testpilot_1,-153.568084,-156.144829
2,testpilot_2,-164.964228,-170.343021
3,testpilot_3,-288.992807,-286.242414
4,testpilot_4,-159.639553,-159.166300
5,testpilot_5,-217.963053,-220.874619
6,testpilot_6,-53.204937,-53.456897
7,testpilot_7,-129.713316,-129.383970
8,testpilot_8,-34.975082,-33.397953
9,testpilot_9,-356.652081,-355.525223
